In [1]:
import os
import re
import mne
import math
import pyedflib
import numpy as np
import pandas as pd

from datetime import datetime, timezone, timedelta
from pyedflib import highlevel, FILETYPE_BDF, FILETYPE_BDFPLUS, FILETYPE_EDF, FILETYPE_EDFPLUS

In [2]:
def excel_to_edf(df, i):
    if not os.path.exists(f'../../data/awake_sleep/jawrhin/analysis/{i}'):
        os.mkdir(f'../../data/awake_sleep/jawrhin/analysis/{i}')
    jawac = [df.Jawac.to_numpy()]
    meas_date = df.Temps[0].to_pydatetime().replace(tzinfo=timezone.utc)
    channel_info = [{'label': 'Jawac', 
                     'dimension': 'mm', 
                     'sample_rate': 10,
                     'physical_min': math.floor(np.min(df.Jawac.to_numpy())), 
                     'physical_max': math.ceil(np.max(df.Jawac.to_numpy())),
                     'digital_min': -32768,
                     'digital_max': 32767,  
                     'transducer': '', 
                     'prefilter': ''}]
    try:
        f = pyedflib.EdfWriter(f'../../data/awake_sleep/jawrhin/analysis/{i}/{i}.edf', n_channels=1)
        f.setSignalHeaders(channel_info)
        f.setStartdatetime(meas_date)
        f.writeSamples(jawac)
    except Exception as e:
        raise e
    finally:
        f.close()

In [3]:
def excel_to_mk3(df, i):
    if not os.path.exists(f'../../data/awake_sleep/jawrhin/analysis/{i}'):
        os.mkdir(f'../../data/awake_sleep/jawrhin/analysis/{i}')
    mk3_file = open(f'../../data/awake_sleep/jawrhin/analysis/{i}/{i}.mk3','w+')
    mk3_file.writelines(['MK_FILE_REV_0\n', 'ANJAW_REV=\n', 'IsReportUpToDate=No\n', 'Comment=\n', '-\n'])
    saved_label = df.Phase[0]
    saved_time = df.Temps[0]
    out_line = []
    for k, v in df.iterrows():
        if v.Phase != saved_label:
            if saved_label == 'Eveil':
                out_line.append(str(saved_time).split(' ')[1] + ':000')
                out_line.append(re.sub(r'(\d{1,2})-(\d{1,2})-(\d{1,2})', '\\3-\\2-\\1', str(saved_time)[2:].split(' ')[0]).replace('-', '.'))
                out_line.append(str(v.Temps).split(' ')[1] + ':000')
                out_line.append(re.sub(r'(\d{1,2})-(\d{1,2})-(\d{1,2})', '\\3-\\2-\\1', str(v.Temps)[2:].split(' ')[0]).replace('-', '.'))
                out_line.append('ApiosUser')
                out_line.append('W')
                mk3_file.write(';'.join(out_line)+'\n')
                out_line = []
            saved_label = v.Phase
            saved_time = v.Temps

In [4]:
print('--- TASK START ---')
for i in range(20):
    df = pd.read_excel('../../data/awake_sleep/jawrhin/data.xlsx', sheet_name = i)
    excel_to_edf(df, i+1)
    excel_to_mk3(df, i+1)
    print(f'--> {i+1} done')
print('--- TASK DONE ---')

--- TASK START ---
--> 1 done
--> 2 done
--> 3 done
--> 4 done
--> 5 done
--> 6 done
--> 7 done
--> 8 done
--> 9 done
--> 10 done
--> 11 done
--> 12 done
--> 13 done
--> 14 done
--> 15 done
--> 16 done
--> 17 done
--> 18 done
--> 19 done
--> 20 done
--- TASK DONE ---
